In [1]:
import os
import sys
import csv
import boto3
import botocore
from retrying import retry

# configuration
s3_bucket = 'adomni-placeiq-sync/neon_query_temp/'  # S3 Bucket name
s3_ouput  = 's3://'+ s3_bucket   # S3 Bucket to store results
database  = 'default'  # The database to which the query belongs

In [2]:
# init clients
athena = boto3.client('athena')
s3 = boto3.resource('s3')

In [3]:
@retry(stop_max_attempt_number = 15,
    wait_exponential_multiplier = 300,
    wait_exponential_max = 1 * 60 * 1000)
def poll_status(_id):
    result = athena.get_query_execution(QueryExecutionId=_id)
    state  = result['QueryExecution']['Status']['State']

    if state == 'SUCCEEDED':
        return result
    elif state == 'FAILED':
        return result
    else:
        raise Exception

In [7]:
def run_query(query, database, s3_output):
    response = athena.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration = {
            'OutputLocation': s3_output,
            'EncryptionConfiguration': {
                'EncryptionOption': 'SSE_S3'
            }
        }
    )
    
    query_execution_id = response['QueryExecutionId']
#     result = athena.get_query_execution(QueryExecutionId=query_execution_id)
    result = poll_status(query_execution_id)
#     print(result)
    
    if result['QueryExecution']['Status']['State'] == 'SUCCEEDED':
        print()
        print("Query SUCCEEDED: ")
        print(query_execution_id)
        
        s3_key = query_execution_id + '.csv'
        filename = query_execution_id + '.csv'
        
        # download result file
#         try:
#             s3.Bucket(s3_bucket).download_file(s3_key, filename)
#         except botocore.exceptions.ClientError as e:
#             if e.response['Error']['Code'] == "404":
#                 print("The object does not exist.")
#             else:
#                 raise
        
#         # read file to array
#         rows = []
#         with open(filename) as csvfile:
#             reader = csv.DictReader(csvfile)
#             for row in reader:
#                 rows.append(row)

#         # delete result file
#         if os.path.isfile(filename):
#             os.remove(filename)

#         return rows
    if result['QueryExecution']['Status']['State'] == 'FAILED':
        print()
        print(result['QueryExecution']['Status']['StateChangeReason'])


In [8]:
# SQL Query to execute

# Database: 
# location_data

# Tables: 
#
# hist_20190817_billboard_audiences: ['billboard_id', 'dma_name', 'audience', 'audience_index']
# 
# hist_20190817_billboard_devices: ['billboard_id', 'mobile_device_id']
# 
# hist_20190817_device_audiences: ['mobile_device_id', 'audience']
# 
# billboard_audiences_partitioned
# billboard_devices_partitioned
# adomni_audience_segment
# current_adomni_map


# query = ("""
#     SELECT * 
#     FROM location_data.hist_20190817_billboard_devices 
#     WHERE mobile_device_id = '2a7e1bff-f551-4ee4-a0d9-16d25f99d75e';
# """)

query = ("""
     select * 
     from location_data.billboard_audiences_partitioned
     where dt = 20190817
     limit 5;
""")

# query = ("""
#      select * 
#      from location_data.hist_20190817_device_audiences
#      limit 100
# """)

# Q1. How many billboards was device '2a7e1bff-f551-4ee4-a0d9-16d25f99d75e' seen in front of?  
# (Answer = 6) What is the query to get this value?
# 
# query = ("""
#       select count(distinct billboard_id)
#       from location_data.hist_20190817_billboard_devices
#       where mobile_device_id = '2a7e1bff-f551-4ee4-a0d9-16d25f99d75e'
#  """)


# Q2. What audiences was the same mobile device seen in?
# 
# query = ("""
#       select distinct audience
#       from location_data.hist_20190817_device_audiences
#       where mobile_device_id = '2a7e1bff-f551-4ee4-a0d9-16d25f99d75e'
#  """)


# Q3. What billboard has the highest number of mobile devices in the "Demographic->Income->200KPlus" segment?
# 
# query = ("""
# select
#   billboard_id, 
#   count(a.mobile_device_id) mycount
# from location_data.hist_20190817_billboard_devices a
# inner join (
#   select 
#     mobile_device_id
#   from 
#     location_data.hist_20190817_device_audiences
#   where audience = 'Demographic->Income->200KPlus' 
# ) b on a.mobile_device_id = b.mobile_device_id  
# group by 
#   billboard_id
# order by
#   mycount desc
# """)


# Q4. What are those device ids?  


# Q5. What other boards do those devices frequent?
#
# query = ("""
# select
#   distinct billboard_id
# from
#   location_data.hist_20190817_billboard_devices
# where
#   mobile_device_id in (
#     select
#       mobile_device_id
#     from
#       location_data.hist_20190817_billboard_devices
#     where
#       billboard_id = 'f2a8fea85d723bd01600c31a307d1e81'
#   )
# """)




print("Executing query: {}".format(query))
result = run_query(query, database, s3_ouput)

# print("Results:")
# print(result)

Executing query: 
     select * 
     from location_data.billboard_audiences_partitioned
     where dt = 20190817
     limit 5;


Query SUCCEEDED: 
01c43b8e-ef0c-4bde-b121-11f02afed241
